# Analysis

## Modules

### Imports

In [2]:
import importlib.metadata
import itertools
import os
import types

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics

### Versions

The current module versions in use are as follows:

In [3]:
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "pillow", "sklearn": "scikit-learn"}
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name


imports = list(set(get_imports()))

requirements = []
for dist in importlib.metadata.distributions():
    if (
        dist.metadata["Name"].lower() in imports
        and dist.metadata["Name"].lower() != "pip"
    ):
        requirements.append((dist.metadata["Name"], dist.version))

pd.DataFrame(requirements, columns=["Module", "Version"])

,Module,Version
0,matplotlib,3.9.2
1,numpy,1.26.4
2,pandas,2.2.3
3,scikit-learn,1.5.2


## Global variables

In [4]:
RESULTS_PATH = "./results"
RAVDESS_TEST_LABELS_PATH = "../nvark-kernel/datasets/KM/RAVDESS/RAVDESS_TEST.tsv"
RAVDESS_EMOTIONS = [
    "neutral",
    "calm",
    "happy",
    "sad",
    "angry",
    "fearful",
    "disgust",
    "surprised",
]
EMODB_TEST_LABELS_PATH = "../nvark-kernel/datasets/KM/EmoDB/EmoDB_TEST.tsv"
EMODB_EMOTIONS = [
    "Anger",
    "Boredom",
    "Disgust",
    "Anxiety/Fear",
    "Happiness",
    "Sadness",
    "Neutral",
]

## Analysis functions

In [5]:
def plot_confusion_matrix(labels, predictions, class_names):
    print(f"Accuracy: {metrics.accuracy_score(labels, predictions) * 100:.3f}%")
    print(
        f"Precision: {metrics.precision_score(labels, predictions, average="macro") * 100:.3f}%"
    )
    print(
        f"Recall: {metrics.recall_score(labels, predictions, average="macro") * 100:.3f}%"
    )

    # cm = tf.math.confusion_matrix(labels=labels, predictions=predictions)
    cm = metrics.confusion_matrix(y_true=labels, y_pred=predictions)

    fig, ax = plt.subplots(figsize=(6, 6))
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=30, ha="right")
    plt.yticks(tick_marks, class_names)

    labels = np.around(
        cm.astype("float") * 100 / cm.sum(axis=1)[:, np.newaxis],
        decimals=2,
    )

    threshold = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(
            j,
            i,
            "\n" + str(int(cm[i, j])) + "\n" + str(labels[i, j]) + "%",
            horizontalalignment="center",
            color=color,
        )

    plt.tight_layout()
    plt.ylabel("True class")
    plt.xlabel("Predicted class")

    plt.show()

## Main

### RAVDESS

#### Train

In [ ]:
train_results = pd.DataFrame(
    np.load(os.path.join(RESULTS_PATH, "RAVDESS_trainmetrics.npy")),
    columns=["Accuracy", "Precision", "Recall", "F1"],
)

In [ ]:
train_results.describe()

#### Test

In [ ]:
labels = (
    pd.read_csv(
        RAVDESS_TEST_LABELS_PATH,
        header=None,
        sep="\t",
        usecols=[0],
    )
    .squeeze()
    .to_numpy()
)

predictions = np.load(os.path.join(RESULTS_PATH, "RAVDESS_testlabels.npy"))

In [ ]:
plot_confusion_matrix(labels, predictions, RAVDESS_EMOTIONS)

### EmoDB

#### Train

In [ ]:
train_results = pd.DataFrame(
    np.load(os.path.join(RESULTS_PATH, "EmoDB_trainmetrics.npy")),
    columns=["Accuracy", "Precision", "Recall", "F1"],
)

In [ ]:
train_results.describe()

#### Test

In [ ]:
labels = (
    pd.read_csv(
        EMODB_TEST_LABELS_PATH,
        header=None,
        sep="\t",
        usecols=[0],
    )
    .squeeze()
    .to_numpy()
)

predictions = np.load(os.path.join(RESULTS_PATH, "EmoDB_testlabels.npy"))

In [ ]:
plot_confusion_matrix(labels, predictions, EMODB_EMOTIONS)